In [1]:
M <- matrix(c(c(-2,-1,2,1),c(3,1,-1,-3),c(-1,0,-1,2)), nrow=4)

In [2]:
pca <- function(M, scale=TRUE, bias=FALSE){
    # returned object :
    base_result = list("base"=M)
    
    # fonction qui calcule la matrice de covariance de la matrice initiale et renvoit toutes les étapes précédentes de calcul
    covariance_matrix <- function(M){
        # fonction qui retourne la matrice des données centrées et le centre de gravité de la matrice initiale
        centered_matrix <- function(M){
            # fonction qui calcule la matrice G, centre de gravité de la matrice passée en paramètre
            gravity_center_matrix <- function(M){
                moy_T <- colMeans(M, na.rm = FALSE, dims = 1)
                M_moy_t <- t(matrix(rep(moy_T, dim(M)[1]), ncol = dim(M)[1]))
                return(M_moy_t)
            }
            G <- gravity_center_matrix(M)
            return(list("G"=G, "M_centered"=M-G))
        }
        Mc <- centered_matrix(M)
        M_temp <- t(Mc[["M_centered"]]) %*% Mc[["M_centered"]]
        S_bias <- (1 / dim(M)[1]) * M_temp
        S <- (1 / (dim(M)[1]-1)) * M_temp
        return(append(Mc, list("S"=S, "S_bias"=S_bias)))
    }
    base_result <- append(base_result, covariance_matrix(M))
    # fonction de calcul de la matrice de correlation de la matrice initiale et renvoit toutes les étapes précédentes de calcul
    correlation_matrix <- function(S, M_centered){
        print(S)
        # fonction qui calcule la matrice diagonale des (1/écarts-type) des colonnes
        diag_ecart_type <- function(S){
            return(list("D1_div_sd"= (1/sqrt(diag(S))) * diag(dim(S)[1])))
        }
        result <- diag_ecart_type(S)
        
        # fonction qui calcule la matrice centrée réduite
        centered_reduced_matrix <- function(M_centered, D1_S){
            return(list("M_scale"= M_centered %*% D1_S))
        }
        result <- append(result, centered_reduced_matrix(M_centered, result[["D1_div_sd"]]))
        
        # calcul de la matrice de correlation
        R_temp <- t(result[["M_scale"]]) %*% result[["M_scale"]]
        R <- (1 / dim(M)[1]) * R_temp
        return(append(result, list("R"=R)))
    }
    if(scale){
        base_result <- append(base_result, correlation_matrix(base_result[["S_bias"]], base_result[["M_centered"]]))
    }
    if(bias){
        base_result[["S"]] <- base_result[["S_bias"]]
    }
    base_result[["S_bias"]] <- NULL
    
    # building PCA main component
    pca_component <- list()
    Si <- c()
    if(scale){
        Si <- base_result[["R"]]
    }else{
        Si <- base_result[["S"]]
    }
    pca_component[["Si"]] <- Si
    
    
    
    return(base_result)
}

In [3]:
print(pca(M, bias=TRUE, scale=TRUE))

     [,1] [,2] [,3]
[1,]  2.5   -3  0.5
[2,] -3.0    5 -2.0
[3,]  0.5   -2  1.5
$base
     [,1] [,2] [,3]
[1,]   -2    3   -1
[2,]   -1    1    0
[3,]    2   -1   -1
[4,]    1   -3    2

$G
     [,1] [,2] [,3]
[1,]    0    0    0
[2,]    0    0    0
[3,]    0    0    0
[4,]    0    0    0

$M_centered
     [,1] [,2] [,3]
[1,]   -2    3   -1
[2,]   -1    1    0
[3,]    2   -1   -1
[4,]    1   -3    2

$S
     [,1] [,2] [,3]
[1,]  2.5   -3  0.5
[2,] -3.0    5 -2.0
[3,]  0.5   -2  1.5

$D1_div_sd
          [,1]      [,2]      [,3]
[1,] 0.6324555 0.0000000 0.0000000
[2,] 0.0000000 0.4472136 0.0000000
[3,] 0.0000000 0.0000000 0.8164966

$M_scale
           [,1]       [,2]       [,3]
[1,] -1.2649111  1.3416408 -0.8164966
[2,] -0.6324555  0.4472136  0.0000000
[3,]  1.2649111 -0.4472136 -0.8164966
[4,]  0.6324555 -1.3416408  1.6329932

$R
           [,1]       [,2]       [,3]
[1,]  1.0000000 -0.8485281  0.2581989
[2,] -0.8485281  1.0000000 -0.7302967
[3,]  0.2581989 -0.7302967  1.0000000



In [4]:
inertia <- function(eigenvalues){
  return(sum(eigenvalues))
}

cumulative_information_caught <- function(eigenvalues, num_axes = 2, percent = T){
  return(sum(eigenvalues[1:num_axes])/inertia(eigenvalues)*((1-(percent*1))+(percent*1)*100))
}

information_caught <- function(eigenvalues, index_axe, percent = T){
  return(eigen_values[index_axe]/inertia(eigenvalues)*((1-(percent*1))+(percent*1)*100))
}

In [126]:
norm <- function(v){
    return(sqrt(sum(v**2)))
}

eigen_function_hh <- function(M){
    QRDecompHH <- function(A){
        T <- A
        I <- diag(dim(A)[1])
        
        vi <- t(A[,1]) - sign(A[1,1])*norm(A[,1])*I[,1]

        
        hi <- I-(t(vi)%*%vi)*matrix(rep(2/vi%*%t(vi), dim(A)[1]*dim(A)[1]), nrow=dim(A)[1], ncol=dim(A)[1])

        
        his <- list(hi)
        Ak <- hi%*%A
        for(i in 2:dim(Ak)[1]){
            if(dim(A)[1] > 2){
                A <- Ak[2:dim(Ak)[1], 2:dim(Ak)[2]]
                #print(A)
                I <- diag(dim(A)[1])

                vi <- t(A[,1]) - sign(A[i,i])*norm(A[,1])*I[,1]
                #print(vi)

                hi <- I-(t(vi)%*%vi)*matrix(rep(2/vi%*%t(vi), dim(A)[1]*dim(A)[1]), nrow=dim(A)[1], ncol=dim(A)[1])
                #print(hi)

                his[[i]] <- hi
                Ak <- hi%*%A
                #print(Ak)
            }
        }
        
        Q <- diag(dim(T)[1])
        R <- diag(dim(T)[1])
        for(i in 1:length(his)){
            I = diag(dim(T)[1])

            c <- his[[i]]
            I[(dim(I)[1]-dim(c)[1]+1):dim(I)[1], (dim(I)[1]-dim(c)[1]+1):dim(I)[1]] <- c
            his[[i]] <- I
            Q <- Q%*%I

        }
        
        
        for(i in 1:length(his)){
            R <- R%*%his[[dim(T)[1]-i]]
        }
        return(list(R%*%T, Q))
    }
    
    eigen_v <- function(A, accuracy){
        Ao <- A
        Uo <- diag(dim(A)[1])
        for(i in 1:accuracy){
            Akk <- QRDecompHH(Ao)
            Ao <- Akk[[1]]%*%Akk[[2]]
            Uo <- Uo%*%Akk[[2]]
        }
        return(list(Uo, Ao))
    }
    
    dimM = dim(M)[1]
    eigen_v_ret <- eigen_v(M, 100)


    return(list("eigen_values"=diag(eigen_v_ret[[2]]), "eigen_vectors"=eigen_v_ret[[1]]))
}

In [130]:
A <- matrix(c(1,2,3,2,4,2,3,2,1), ncol=3, nrow=3, byrow=T)

A
e = eigen_function(A)
sum(A)

1,2,3
2,4,2
3,2,1


[[1]]
          [,1]          [,2]       [,3]
[1,] 0.5000000 -7.071068e-01  0.5000000
[2,] 0.7071068 -8.931017e-10 -0.7071068
[3,] 0.5000000  7.071068e-01  0.5000000

[[2]]
              [,1]          [,2]          [,3]
[1,]  6.828427e+00 -1.114902e-08 -7.289442e-12
[2,] -1.114902e-08 -2.000000e+00  5.766347e-13
[3,] -7.289965e-12  5.770261e-13  1.171573e+00



ERROR: Error in eigen_function(A): objet 'eigen_values' introuvable


In [131]:
eigen(A)

eigen() decomposition
$values
[1]  6.828427  1.171573 -2.000000

$vectors
           [,1]       [,2]          [,3]
[1,] -0.5000000  0.5000000  7.071068e-01
[2,] -0.7071068 -0.7071068 -1.554312e-15
[3,] -0.5000000  0.5000000 -7.071068e-01


In [69]:
t = A-e$eigen_values[2]*diag(2)
t[2,]%*%e$eigen_vectors[,2]

-0.9093767


In [40]:
I <- diag(3)
I

1,0,0
0,1,0
0,0,1


In [41]:
c <- matrix(c(1,2,3,4), nrow = 2, ncol = 2)
c

1,3
2,4


In [43]:
I[(dim(I)[1]-dim(c)[1]+1):dim(I)[1], (dim(I)[1]-dim(c)[1]+1):dim(I)[1]] <- c

In [44]:
I

1,0,0
0,1,3
0,2,4


In [45]:
dim(I)[1]-dim(c)[1]+1

[1] 2